In [1]:
import numpy as np 
import pandas as pd 

In [2]:
# train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
# test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')
train = pd.read_csv('train.csv', parse_dates=['Dates'])
test = pd.read_csv('test.csv', parse_dates=['Dates'], index_col='Id')

In [3]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
test.head()

,Dates,DayOfWeek,PdDistrict,Address,X,Y
Id,,,,,,
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [5]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train["Address"]) + list(test["Address"]))

train_seq = tokenizer.texts_to_sequences(train["Address"])
test_seq = tokenizer.texts_to_sequences(test["Address"])

from keras.preprocessing.sequence import pad_sequences
train_seq = pad_sequences(train_seq, maxlen=7)
test_seq = pad_sequences(test_seq, maxlen=7)

from keras import Sequential
from keras.layers import Flatten, Embedding, Dense 
model = Sequential()
model.add(Embedding(2201, 1, input_length=7))
model.add(Flatten())
model.add(Dense(39, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

Using TensorFlow backend.


In [6]:
from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])
model.fit(train_seq, y, epochs = 10 , batch_size = 2048)

train_preds = model.predict(train_seq)
test_preds = model.predict(test_seq)

C:\Users\Jiyoung Sim\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
878049/878049 [==============================] - 2s 2us/step - loss: 3.0936 - acc: 0.1349
Epoch 2/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.6540 - acc: 0.2038
Epoch 3/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.6184 - acc: 0.2103
Epoch 4/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.6035 - acc: 0.2102
Epoch 5/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5942 - acc: 0.2104
Epoch 6/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5869 - acc: 0.2136
Epoch 7/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5810 - acc: 0.2156
Epoch 8/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5763 - acc: 0.2197
Epoch 9/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5726 - acc: 0.2225
Epoch 10/10
878049/878049 [==============================] - 2s 2us/step - loss: 2.5698 - a

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
train_addr = pca.fit_transform(train_preds)
test_addr = pca.transform(test_preds)

In [8]:
train['Date'] = pd.to_datetime(train['Dates'].dt.date)
train['n_days'] = (train['Date'] - train['Date'].min()).apply(lambda x: x.days)
train['Day'] = train['Dates'].dt.day
train['DayOfWeek'] = train['Dates'].dt.weekday
train['Month'] = train['Dates'].dt.month
train['Year'] = train['Dates'].dt.year
train['Hour'] = train['Dates'].dt.hour
train['Minute'] = train['Dates'].dt.minute

test['Date'] = pd.to_datetime(test['Dates'].dt.date)
test['n_days'] = (test['Date'] - test['Date'].min()).apply(lambda x: x.days)
test['Day'] = test['Dates'].dt.day
test['DayOfWeek'] = test['Dates'].dt.weekday
test['Month'] = test['Dates'].dt.month
test['Year'] = test['Dates'].dt.year
test['Hour'] = test['Dates'].dt.hour
test['Minute'] = test['Dates'].dt.minute

In [9]:
train['X-Y'] = train['X'] - train['Y']
train["X+Y"] = train['X'] + train['Y']

test['X-Y'] = test['X'] - test['Y']
test['X+Y'] = test['X'] + test['Y']

In [10]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
y= le1.fit_transform(train['Category'])

le2 = LabelEncoder()
train['PdDistrict'] = le2.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le2.transform(test['PdDistrict'])

train = pd.concat([train, pd.DataFrame(train_addr)], 1)
test = pd.concat([test, pd.DataFrame(test_addr)], 1)

In [11]:
import re
train['Address'] = train['Address'].apply(lambda x: re.sub('[0-9]+ Block of ', '', x, re.IGNORECASE))
test['Address'] = test['Address'].apply(lambda x: re.sub('[0-9]+ Block of ', '', x, re.IGNORECASE))

In [12]:
le3 = LabelEncoder()
le3.fit(list(train['Address']) + list(test['Address']))
train['Address'] = le3.transform(train['Address'])
test['Address'] = le3.transform(test['Address'])

In [13]:
train.drop(['Dates','Date','Descript','Resolution', 'Category'], 1, inplace=True)
test.drop(['Dates','Date',], 1, inplace=True)

In [14]:
from lightgbm import LGBMClassifier
hyper = {'colsample_bytree': 0.6,
 'is_unbalance': False,
 'learning_rate': 0.4,
 'min_data_in_leaf': 100,
 'num_class': 39,
 'num_leaves': 50,
 'max_delta_step' : 0.9,
 'objective': 'multiclass',
 'reg_alpha': 0.4,
 'reg_lambda': 0.5,
 'bagging_fraction': 0.9,
 'n_estimators': 100} # max_bin = 255, bin_construct_sample_cnt = 200000
model = LGBMClassifier(**hyper)
model.fit(train, y, categorical_feature=["PdDistrict", "DayOfWeek"])
preds = model.predict_proba(test)

C:\Users\Jiyoung Sim\Anaconda3\lib\site-packages\lightgbm\basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DayOfWeek', 'PdDistrict']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [15]:
submission = pd.DataFrame(preds, columns=le1.inverse_transform(np.linspace(0, 38, 39, dtype='int16')), index=test.index)
submission.to_csv('sf-crime-lgbm2.csv', index_label='Id')